In [2]:
! pip install pyLDAvis

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 1.7/1.7 MB 869.3 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 92.7/92.7 kB 1.1 MB/s eta 0:00:00
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136904 sha256=a4d4b3352cdd4b8dd6add2719cfdca094de5a370acaa5aef0faa51924f8d52a0
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\1f\ef\46\1dc1c1e5d8ff2f3b44cccaf004cf64488cf82b5c359fbc8efb
Successfully built pyLDAvis



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
! pip install nlp

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 1.7/1.7 MB 857.6 kB/s eta 0:00:00
     -------------------------------------- 20.0/20.0 MB 929.1 kB/s eta 0:00:00
     -------------------------------------- 110.5/110.5 kB 1.1 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import re
import nltk
import spacy
import gensim
import pyLDAvis
import nlp
nltk.download('stopwords')
import pandas as pd
import numpy as np

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for Lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df = pd.read_csv('bbc-text.csv')
df.shape

(2225, 2)

In [7]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [10]:
# convert to list
data = df['text'].values.tolist()

# remove emails
data = [re.sub('\S*@\S*\s?', "", i) for i in data]

# remove new line characters
data = [re.sub('\s+',' ', i) for i in data]

# remove distracting single quotes
data = [re.sub("\'", "", i) for i in data]

<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
C:\Users\User\AppData\Local\Temp\ipykernel_26416\3303659828.py:5: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', "", i) for i in data]
C:\Users\User\AppData\Local\Temp\ipykernel_26416\3303659828.py:8: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+',' ', i) for i in data]


In [12]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) # deacc=True removes punctuation

data_words = list(sent_to_words(data))

print(data_words[:1])

[['tv', 'future', 'in', 'the', 'hands', 'of', 'viewers', 'with', 'home', 'theatre', 'systems', 'plasma', 'high', 'definition', 'tvs', 'and', 'digital', 'video', 'recorders', 'moving', 'into', 'the', 'living', 'room', 'the', 'way', 'people', 'watch', 'tv', 'will', 'be', 'radically', 'different', 'in', 'five', 'years', 'time', 'that', 'is', 'according', 'to', 'an', 'expert', 'panel', 'which', 'gathered', 'at', 'the', 'annual', 'consumer', 'electronics', 'show', 'in', 'las', 'vegas', 'to', 'discuss', 'how', 'these', 'new', 'technologies', 'will', 'impact', 'one', 'of', 'our', 'favourite', 'pastimes', 'with', 'the', 'us', 'leading', 'the', 'trend', 'programmes', 'and', 'other', 'content', 'will', 'be', 'delivered', 'to', 'viewers', 'via', 'home', 'networks', 'through', 'cable', 'satellite', 'telecoms', 'companies', 'and', 'broadband', 'service', 'providers', 'to', 'front', 'rooms', 'and', 'portable', 'devices', 'one', 'of', 'the', 'most', 'talked', 'about', 'technologies', 'of', 'ces', 'ha

In [14]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

def lemmatization(texts, allowed_postags=['NOUN']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [15]:
# remove stop words
data_words_nostops = remove_stopwords(data_words)

In [16]:
# do lemmatization keeping only noun
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN'])

In [17]:
print(data_lemmatized[:2])

[['tv', 'viewer', 'home', 'theatre', 'system', 'definition', 'video', 'recorder', 'living', 'room', 'way', 'people', 'tv', 'year', 'time', 'expert', 'panel', 'consumer', 'electronic', 'show', 'technology', 'impact', 'pastime', 'trend', 'programme', 'content', 'viewer', 'home', 'network', 'cable', 'satellite', 'telecom', 'company', 'service', 'provider', 'room', 'device', 'technology', 'video', 'recorder', 'dvr', 'box', 'sky', 'system', 'people', 'record', 'store', 'pause', 'wind', 'tv', 'programme', 'technology', 'tv', 'definition', 'tv', 'business', 'definition', 'programming', 'people', 'wind', 'advert', 'network', 'channel', 'schedule', 'carte', 'network', 'cable', 'satellite', 'company', 'term', 'advertising', 'revenue', 'brand', 'identity', 'viewer', 'loyalty', 'channel', 'technology', 'moment', 'concern', 'uptake', 'service', 'sky', 'today', 'month', 'year', 'futurologist', 'news', 'website', 'issue', 'advertising', 'revenue', 'issue', 'moment', 'brand', 'loyalty', 'content', 'br

In [18]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
print(id2word)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

Dictionary<10584 unique tokens: ['advert', 'advertising', 'bill', 'book', 'box']...>


In [19]:
# baseline model

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=5,
                                            random_state=100,
                                            chunksize=200,
                                            passes=10,
                                            per_word_topics=True)

In [20]:
# print the keyword in the 5 topics
from pprint import pprint
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.022*"player" + 0.021*"year" + 0.020*"game" + 0.019*"time" + 0.016*"team" '
  '+ 0.013*"club" + 0.012*"side" + 0.011*"match" + 0.010*"week" + '
  '0.010*"manager"'),
 (1,
  '0.039*"film" + 0.021*"year" + 0.018*"music" + 0.015*"award" + '
  '0.013*"domain" + 0.011*"album" + 0.010*"number" + 0.010*"show" + '
  '0.009*"band" + 0.009*"song"'),
 (2,
  '0.032*"game" + 0.011*"woman" + 0.010*"prisoner" + 0.009*"year" + '
  '0.009*"time" + 0.008*"world" + 0.008*"case" + 0.007*"man" + 0.007*"school" '
  '+ 0.007*"arm"'),
 (3,
  '0.025*"people" + 0.012*"phone" + 0.012*"attack" + 0.012*"technology" + '
  '0.011*"service" + 0.011*"computer" + 0.010*"network" + 0.010*"site" + '
  '0.009*"software" + 0.009*"user"'),
 (4,
  '0.020*"year" + 0.015*"government" + 0.012*"sale" + 0.010*"party" + '
  '0.009*"election" + 0.009*"company" + 0.009*"people" + 0.008*"country" + '
  '0.008*"plan" + 0.007*"labour"')]


In [21]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4498515665457409


In [23]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
LDAvis_prepared

C:\Users\User\AppData\Roaming\Python\Python39\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.188249 -0.077773       1        1  36.051164
3      0.183917  0.115195       2        1  23.740322
0     -0.129508 -0.128281       3        1  14.469757
1     -0.155298  0.224051       4        1  13.637092
2     -0.087361 -0.133192       5        1  12.101666, topic_info=           Term         Freq        Total Category  logprob  loglift
240        film  1071.000000  1071.000000  Default  30.0000  30.0000
166        game  1290.000000  1290.000000  Default  29.0000  29.0000
537      player   781.000000   781.000000  Default  28.0000  28.0000
297  government  1042.000000  1042.000000  Default  27.0000  27.0000
435       music   747.000000   747.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
98         time   192.882697  1492.856255   Topic5  -4.7646   0.0655
112        year   207.878702  2909.759616   Topic5  -4.6897  -0.5270
523       court    99.134140   212.457530   Topic5  -5.4302   1.3496
109         way   127.200772   776.249440   Topic5  -5.1809   0.3031
69       people   151.455657  1998.688294   Topic5  -5.0064  -0.4681

[308 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
2295      4  0.971737  academy
785       1  0.283041   action
785       2  0.101086   action
785       3  0.111195   action
785       4  0.043804   action
...     ...       ...      ...
112       1  0.457082     year
112       2  0.095197     year
112       3  0.195205     year
112       4  0.181115     year
112       5  0.071484     year

[494 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 1, 2, 3])

In [24]:
# For topic distribution across documents and wordcloud, refer to https://www.linkedin.com/pulse/nlp-a-complete-guide-topic-modeling-latent-dirichlet-sahil-m/